# Training AID dataset by SRCNN

- reference code: https://github.com/worldstrat/worldstrat.git

In [6]:
%load_ext autoreload
%autoreload 2
from src.train import *
import sys

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [ ]:
default_train_command = [
    # Batch size, gpus, limits
    "train.py",
    "--batch_size", "1", 
    "--accelerator", "cpu",
    "--devices", "1",
    "--max_steps", "100",
    "--precision", "16",
    "--num_workers", "0",

    # Model/Hyperparameters
    "--model", "SRCNN", # must be uppercase
    "--w_mse", "0.3",
    "--w_mae", "0.4",
    "--w_ssim", "0.3",
    "--hidden_channels", "32",
    "--residual_layers", "1",
    "--padding_mode", "reflect",
    "--sr_kernel_size", "1",
    "--use_dropout", "False",
    "--use_batchnorm", "False",
    "--learning_rate", "1e-4",

    # Data
    "--root", "AID-dataset/",
    "--zoom_factor", "2", # 모델에 영향
    "--output_size", "300", "300",
    "--chip_size", "300", "300",
    "--chip_stride", "300", "300",
    "--randomly_rotate_and_flip_images", "True",
    "--shuffle", "True",
    "--subset_train", "0.6",

    # logging
    "--use_wandb",
    # "--benchmark_logging", # require enable wandb

    "--upload_checkpoint",

    # early stopping
    "--early_stop",
    "--early_stop_patience", "8",
    "--early_stop_min_delta", "0.001",
]

def run_training_command(training_command, running_on_windows=True):
    sys.argv = training_command
    if running_on_windows:
        sys.argv += ["--num_workers", "0"] # 윈도우에서는 멀티프로세싱 비효율(than linux)
    cli_main()

In [ ]:
run_training_commanㅇㅇd(default_train_command, running_on_windows=False)

Global seed set to 1337


train_lr dataset path: AID-dataset/train/LR_x4
train_hr dataset path: AID-dataset/train/HR
val_lr dataset path: AID-dataset/val/LR_x4
val_hr dataset path: AID-dataset/val/HR
test_lr dataset path: AID-dataset/test/LR_x4
test_hr dataset path: AID-dataset/test/HR
Shuffling the dataset splits using 42
Shuffling the dataset splits using 42
Shuffling the dataset splits using 42
Shuffling the dataset splits using 42
Train set size: 4710
Val set size: 150
Test set size: 2000


/opt/homebrew/anaconda3/envs/worldstrat/lib/python3.9/site-packages/pytorch_lightning/utilities/parsing.py:263: UserWarning: Attribute 'backbone' is an instance of `nn.Module` and is already saved during checkpointing. It is recommended to ignore them using `self.save_hyperparameters(ignore=['backbone'])`.
  rank_zero_warn(
/opt/homebrew/anaconda3/envs/worldstrat/lib/python3.9/site-packages/pytorch_lightning/trainer/connectors/accelerator_connector.py:736: UserWarning: You passed `Trainer(accelerator='cpu', precision=16)` but native AMP is not supported on CPU. Using `precision='bf16'` instead.
  rank_zero_warn(
Using bfloat16 Automatic Mixed Precision (AMP)
Trainer already configured with model summary callbacks: [<class 'pytorch_lightning.callbacks.model_summary.ModelSummary'>]. Skipping setting a default `ModelSummary` callback.
GPU available: True (mps), used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
/opt


===== Model Backbone Architecture =====
SRCNN(
  (encoder): DoubleConv2d(
    (doubleconv2d): Sequential(
      (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=same, bias=False, padding_mode=reflect)
      (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): PReLU(num_parameters=1)
      (3): Dropout(p=0.5, inplace=False)
      (4): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=same, bias=False, padding_mode=reflect)
      (5): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (6): PReLU(num_parameters=1)
      (7): Dropout(p=0.5, inplace=False)
    )
  )
  (body): Sequential(
    (0): ResidualBlock(
      (residualblock): DoubleConv2d(
        (doubleconv2d): Sequential(
          (0): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=same, bias=False, padding_mode=reflect)
          (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
     

Sanity Checking: 0it [00:00, ?it/s]

/opt/homebrew/anaconda3/envs/worldstrat/lib/python3.9/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:224: PossibleUserWarning: The dataloader, val_dataloader 0, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 10 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  rank_zero_warn(
/opt/homebrew/anaconda3/envs/worldstrat/lib/python3.9/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:224: PossibleUserWarning: The dataloader, train_dataloader, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 10 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  rank_zero_warn(


Training: 0it [00:00, ?it/s]

/opt/homebrew/anaconda3/envs/worldstrat/lib/python3.9/site-packages/pytorch_lightning/trainer/call.py:48: UserWarning: Detected KeyboardInterrupt, attempting graceful shutdown...
  rank_zero_warn("Detected KeyboardInterrupt, attempting graceful shutdown...")
/opt/homebrew/anaconda3/envs/worldstrat/lib/python3.9/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:224: PossibleUserWarning: The dataloader, test_dataloader 0, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 10 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  rank_zero_warn(


Testing: 0it [00:00, ?it/s]

RuntimeError: Broken pipe

### Training a single model

In [ ]:
run_training_command(default_train_command, running_on_windows=False)

Global seed set to 1337


train_lr dataset path: AID-dataset/train/LR_x2
train_hr dataset path: AID-dataset/train/HR
val_lr dataset path: AID-dataset/val/LR_x2
val_hr dataset path: AID-dataset/val/HR
test_lr dataset path: AID-dataset/test/LR_x2
test_hr dataset path: AID-dataset/test/HR


/opt/homebrew/anaconda3/envs/worldstrat/lib/python3.9/site-packages/pytorch_lightning/utilities/parsing.py:263: UserWarning: Attribute 'backbone' is an instance of `nn.Module` and is already saved during checkpointing. It is recommended to ignore them using `self.save_hyperparameters(ignore=['backbone'])`.
  rank_zero_warn(
wandb: WARNING `wandb.require('service')` is a no-op as it is now the default behavior.
wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize?ref=models
wandb: Paste an API key from your profile and hit enter:

Shuffling the dataset splits using 42
Shuffling the dataset splits using 42
Shuffling the dataset splits using 42
Shuffling the dataset splits using 42
Train set size: 4710
Val set size: 150
Test set size: 2000

===== Model Backbone Architecture =====
SRCNN(
  (encoder): DoubleConv2d(
    (doubleconv2d): Sequential(
      (0): Conv2d(3, 32, kernel_size=(3, 3), stride=(1, 1), padding=same, bias=False, padding_mode=reflect)
      (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): PReLU(num_parameters=1)
      (3): Dropout(p=0.5, inplace=False)
      (4): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=same, bias=False, padding_mode=reflect)
      (5): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (6): PReLU(num_parameters=1)
      (7): Dropout(p=0.5, inplace=False)
    )
  )
  (body): Sequential(
    (0): ResidualBlock(
      (residualblock): DoubleConv2d(
        (doubleconv2d): Sequential(
  